In [1]:
import bz2 
import pickle
import numpy as np 
from pathlib import Path
from transformers import MusicgenProcessor, AutoProcessor, MusicgenForConditionalGeneration

/home/rhw/.pyenv/versions/3.10.12/envs/musicgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor: MusicgenProcessor = AutoProcessor.from_pretrained("facebook/musicgen-small")
full_model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")


In [5]:
with bz2.BZ2File("generated_train_dataset/sample-0-0.bz") as bf:
    data1 = pickle.load(bf)
with bz2.BZ2File("generated_train_dataset/sample-1-0-2023-11-19 16:56:18.662404.bz") as bf:
    data2 = pickle.load(bf)

In [6]:
for k, v in data1.items():
    print(k, v.shape)
print()
for k, v in data2.items():
    print(k, v.shape)

input_ids (4, 1500)
attention_mask (31,)
encoder_hidden_states (31, 1024)
labels (4, 1500)
padding_mask (960000,)

input_ids (4, 1500)
attention_mask (24,)
encoder_hidden_states (24, 1024)
labels (4, 1500)
padding_mask (960000,)


In [38]:
def pad_sequences(samples, pad_value):
    max_len = max(sample.shape[0] for sample in samples)
    for idx, sample in enumerate(samples):
        res = np.zeros((max_len, *sample.shape[1:]), dtype=sample.dtype) + pad_value
        res[sample.shape[0]:] = sample 
        samples[idx] = sample 
    return samples 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
padded = processor.tokenizer.pad([data1, data2], max_length=31)
for k, v in padded.items():
    print(k, v.shape)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids (2, 4, 1500)
attention_mask (2,)
encoder_hidden_states (2,)
labels (2, 4, 1500)
padding_mask (2, 960000)


/home/rhw/.pyenv/versions/3.10.12/envs/musicgen/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [56]:
import torch


padded["attention_mask"]
torch.nn.utils.rnn.pad_sequence([torch.from_numpy(padded["attention_mask"][0]), torch.from_numpy(padded["attention_mask"][1])], batch_first=True, padding_value=0)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

In [51]:
padded["encoder_hidden_states"][0][-3:]

array([[-0.,  0.,  0., ..., -0., -0., -0.],
       [-0.,  0.,  0., ..., -0., -0., -0.],
       [-0.,  0.,  0., ..., -0., -0., -0.]])

In [ ]:
np.pad([padded["attention_mask"][0], padded["attention_mask"][1]], pad_width=55)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [31]:
data2["encoder_hidden_states"].shape

(24, 1024)

In [34]:
del padded["padding_mask"]

In [37]:
import torch


with torch.inference_mode():
    padded = {k: torch.from_numpy(v) for k, v in padded.items()}
    full_model.to("cpu")
    full_model.decoder(**padded)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [3]:
# input_ids - from encodec 
# attention_mask - from tokenizer 
# encoder_hidden_states - from tokenizer
# labels - from encodec 
# padding_mask - from encodec prep


dict_keys(['input_ids', 'attention_mask', 'encoder_hidden_states', 'labels', 'padding_mask'])

In [4]:
data1["labels"]

array([[1421,  895, 1981, ..., 1681, 1021,  255],
       [1355, 1400,  522, ..., 1325,  539,  185],
       [ 620, 1883,  839, ..., 1461,  509,  104],
       [  84,   89,  917, ...,  543, 1223,  447]])

In [5]:
data1["input_ids"]

array([[2048, 1421,  895, ...,  845, 1681, 1021],
       [2048, 1355, 1400, ..., 1193, 1325,  539],
       [2048,  620, 1883, ...,  959, 1461,  509],
       [2048,   84,   89, ..., 1447,  543, 1223]])

In [7]:
data1["decoder_input_ids"][0]

array([2048, 2048, 2048, ..., 2048, 2048, 2048])

In [8]:
is_same = []
max_len = 0
for file in Path("generated_train_dataset").iterdir():
    with bz2.BZ2File(file, "rb") as df:
        data = pickle.load(df)
        curr_len = data["attention_mask"].shape[0]
        max_len = max(max_len, curr_len)
        print(curr_len)       
        # is_same.append(
        #     data["decoder_input_ids"][0] == data1["decoder_input_ids"][0]
        # )

19
47
43
41
26
20
19
19
20
54
38
20
20
16
38
19
31
19
46
30
41
17
28
18
44
20
38
18
41
20
17
38
42
42
16
19
29
49
18
20
43
29
42
32
53
18
28
22
30
21
33
20
27
20
38
39
24
16
30
16
30
21
22
30
33
16
34
24
32
17
16
40
18
42
20
28
39
28
27
19
34
46
40
18
17
32
50
37
37
40
22
33
34
23
20
32
20
20
32
39
27
25
20
32
36
44
28
19
29
22
34
29
35
28
43
34
50
53
33
16
38
20
20
38
31
18
34
20
39
21
17
19
55
20
19
34
44
31
33
21
16
42
20
33
19
37
53
21
44
45
21
38
21
31
41
44
45
31
22
20
27
19
44
36
20
17
34
22
40
16
43
38
19
18
44
33
47
33
20
20
36
29
34
20
20
29
21
38
17
29
32
32
50
19
22
39
19
25
47
36
34
20
38
20
19
19
20
32
29
23
43
26
17
39
27
30
41
47
21
32
36
21
20
34
33
35
20
33
16
17
35
44
37
20
32
20
20
19
41
16
29
19
31
22
27
50
34
44
33
44
42
19
41
34
19
37
20
33
50
20
25
43
19
47
33
20
50
34
16
32
34
46
41
16
19
51
20
32
20
51
20
37
46
25
20
16
32
44
21
42
17
38
19
24
36
37
19
19
48
35
33
49
38
19
37
19
27
20
19
40
20
19
43
18
26
26
28
35
28
32
21
46
19
42
43
19
27
44
23
17
27
39
37
1

In [13]:
all(map(lambda x: x.all(), is_same))

True

In [9]:
print(max_len)

57
